In [48]:
import pytesseract
import cv2
import sys
import pandas as pd
import numpy as np
import os
import re
from scipy import ndimage

In [14]:
def rotatationCheck(im):
    if int(re.search('(?<=Rotate: )\d+', pytesseract.image_to_osd(im)).group(0)) !=0:
        #rotation angle in degree
        rotated = ndimage.rotate(im, int(re.search('(?<=Rotate: )\d+', pytesseract.image_to_osd(im)).group(0)))
        return rotated
    else:
        return im

In [25]:
def rotateImage(image):
    angle=int(re.search('(?<=Rotate: )\d+', pytesseract.image_to_osd(image)).group(0))
    row,col = image.shape
    center=tuple(np.array([row,col])/2)
    rot_mat = cv2.getRotationMatrix2D(center,360-angle,1.0)
    new_image = cv2.warpAffine(image, rot_mat, (col,row))
    return new_image

In [35]:
def rotate(image, center = None, scale = 1.0):
    angle=360-int(re.search('(?<=Rotate: )\d+', pytesseract.image_to_osd(image)).group(0))
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated

In [36]:
def imagePreprocess(img):
    # Rescale the image, if needed.
    img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)
    # Convert to gray
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply dilation and erosion to remove some noise
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    # Apply threshold to get image with only black and white
    img = cv2.adaptiveThreshold(cv2.medianBlur(img, 3), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 2)
    
    return img

In [45]:
def images_to_text(testImg):
    print('Reading images form the directory..........')
    dataFile=[]
    for filename in os.listdir(testImg):
        os.chdir(testImg)
        # Define config parameters.
        # '-l eng'  for using the English language 
        # '--oem 1' for using LSTM OCR Engine
        config = ('-l eng --oem 1 --psm 3')
        # Read image from disk
         # Read image from disk
        im = cv2.imread(str(filename), cv2.IMREAD_COLOR)
        # Apply preprocessing on image
        #im = imagePreprocess(im)
        # performing rotation check
        #im = rotatationCheck(image)
        #im=rotateImage(image)
        im=rotate(im)
        # Run tesseract OCR on image
        text = pytesseract.image_to_string(im, config=config)
        # getting hocr output
        #pytesseract.run_tesseract('image.png', 'output', lang=None, boxes=False, config="hocr")
        #basic preprocessing of the text
        text = text.replace('\t',' ')
        text= text.rstrip()
        text= text.lstrip()
        text = text.replace(' +',' ')
        text = text.replace('\n+','\n')
        text = text.replace('\n+ +',' ')

        #writing data to file
        os.chdir(imgTxt)
        rep=filename[-3:]
        name=filename.replace(rep,'txt')
        with open(name, 'w') as writeFile:
            writeFile.write("%s\n" % text)
        text = text.replace('\n',' ')
        dataFile.append(text)
    print('Writing data to file done')    
    return dataFile

In [46]:
testImg="/home/ikscare/Documents/Projects/Mousam/imageFile"
imgTxt="/home/ikscare/Documents/Projects/Mousam/DC_using_OCR/imageTxt"
images = []

for filename in os.listdir(testImg):
	img = os.path.join(testImg,filename)
	if img is not None:
		images.append(img)

#readContent=images_to_text(images)
readContent=images_to_text(testImg)
print("Done")
print('Total number of images read and converted: ',len(readContent))

Reading images form the directory..........
Writing data to file done
Done
Total number of images read and converted:  9


In [56]:
from pytesseract import pytesseract
image='/home/ikscare/Documents/Projects/Mousam/imageFile/explanation_of_benefits.txt-page0.tiff'
pytesseract.run_tesseract(image, '/home/ikscare/Documents/Projects/Mousam/imageFile/output', lang=None, extension='tiff', config="hocr")

True